# 构建语义搜索引擎

本教程将使您熟悉 LangChain 的 [文档加载器](/docs/concepts/document_loaders)、[嵌入](/docs/concepts/embedding_models) 和 [向量存储](/docs/concepts/vectorstores) 抽象。这些抽象旨在支持从（向量）数据库和其他来源检索数据，以便与 LLM 工作流集成。它们对于在模型推理过程中获取数据进行推理的应用程序非常重要，例如检索增强生成或 [RAG](/docs/concepts/rag)（请参阅我们的 RAG 教程 [此处](/docs/tutorials/rag)）。

在这里，我们将构建一个 PDF 文档的搜索引擎。这将使我们能够检索与输入查询相似的 PDF 中的段落。

## 概念

本指南重点介绍文本数据的检索。我们将涵盖以下概念：

- 文档和文档加载器；
- 文本分割器；
- 嵌入；
- 向量存储和检索器。

## 设置

### Jupyter Notebook

本教程和其他教程可能最方便在 Jupyter notebook 中运行。请参阅 [此处](https://jupyter.org/install) 了解安装说明。

### 安装

本教程需要 `langchain-community` 和 `pypdf` 包：

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain-community pypdf</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain-community pypdf -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>


有关更多详细信息，请参阅我们的 [安装指南](/docs/how_to/installation)。

### LangSmith

您使用 LangChain 构建的许多应用程序将包含多个步骤和多次 LLM 调用。
随着这些应用程序变得越来越复杂，能够检查链或代理内部发生的确切情况变得至关重要。
最好的方法是使用 [LangSmith](https://smith.langchain.com)。

在上面的链接注册后，请确保设置环境变量以开始记录跟踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

或者，如果在 notebook 中，您可以使用以下方式设置它们：

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```


## 文档和文档加载器

LangChain 实现了一个 [文档](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) 抽象，旨在表示一个文本单元及其相关元数据。它有三个属性：

- `page_content`：表示内容的字符串；
- `metadata`：包含任意元数据的字典；
- `id`：（可选）文档的字符串标识符。

`metadata` 属性可以捕获有关文档来源、与其他文档的关系以及其他信息。请注意，单个 `Document` 对象通常表示较大文档的一个片段。

我们可以在需要时生成示例文档：
```python
from langchain_core.documents import Document

documents = [
    Document(
        page_content="狗是很好的伴侣，以忠诚和友好著称。",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="猫是独立的宠物，通常喜欢自己的空间。",
        metadata={"source": "mammal-pets-doc"},
    ),
]
```

然而，LangChain 生态系统实现了 [文档加载器](/docs/concepts/document_loaders)，它 [与数百个常见来源集成](/docs/integrations/document_loaders/)。这使得将这些来源的数据集成到您的 AI 应用程序中变得容易。

### 加载文档

让我们将一个 PDF 加载到一系列 `Document` 对象中。在 LangChain 仓库中有一个示例 PDF [此处](https://github.com/langchain-ai/langchain/tree/master/docs/docs/example_data) —— 2023 年 Nike 的 10-k 文件。我们可以查阅 LangChain 文档以了解 [可用的 PDF 文档加载器](/docs/integrations/document_loaders/#pdfs)。让我们选择 [PyPDFLoader](/docs/integrations/document_loaders/pypdfloader/)，它相当轻量级。

In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../example_data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


:::提示

请参阅 [此指南](/docs/how_to/document_loader_pdf/) 以了解有关 PDF 文档加载器的更多详细信息。

:::

`PyPDFLoader` 每个 PDF 页面加载一个 `Document` 对象。对于每个对象，我们可以轻松访问：

- 页面内容的字符串；
- 包含文件名和页码的元数据。

In [2]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FO

{'source': '../example_data/nke-10k-2023.pdf', 'page': 0}


### 分割

对于信息检索和下游问答目的，页面可能是一个过于粗略的表示。我们的最终目标是检索回答输入查询的 `Document` 对象，进一步分割我们的 PDF 将有助于确保文档相关部分的含义不会被周围文本“冲淡”。

我们可以使用 [文本分割器](/docs/concepts/text_splitters) 来实现这一目的。在这里，我们将使用一个简单的基于字符的文本分割器。我们将文档分割成 1000 个字符的块，每块之间有 200 个字符的重叠。重叠有助于缓解将声明与相关上下文分离的可能性。我们使用 [RecursiveCharacterTextSplitter](/docs/how_to/recursive_text_splitter)，它将使用常见分隔符（如换行符）递归分割文档，直到每个块达到适当的大小。这是推荐用于通用文本用例的文本分割器。

我们设置 `add_start_index=True`，以便在初始文档中每个分割文档开始的字符索引作为元数据属性“start_index”保留。

请参阅 [此指南](/docs/how_to/document_loader_pdf/) 以了解有关处理 PDF 的更多详细信息，包括如何从特定部分和图像中提取文本。

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

514

## 嵌入

向量搜索是一种常见的存储和搜索非结构化数据（例如非结构化文本）的方法。其思想是存储与文本相关联的数值向量。给定一个查询，我们可以将其 [嵌入](/docs/concepts/embedding_models) 为相同维度的向量，并使用向量相似性度量（例如余弦相似性）来识别相关文本。

LangChain 支持来自 [多个提供商](/docs/integrations/text_embedding/) 的嵌入。这些模型指定了文本应如何转换为数值向量。让我们选择一个模型：

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs customVarName="embeddings" />

In [3]:
# | output: false
# | echo: false

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"生成长度为 {len(vector_1)} 的向量\n")
print(vector_1[:10])

Generated vectors of length 1536

[-0.008586574345827103, -0.03341241180896759, -0.008936782367527485, -0.0036674530711025, 0.010564599186182022, 0.009598285891115665, -0.028587326407432556, -0.015824200585484505, 0.0030416189692914486, -0.012899317778646946]


有了生成文本嵌入的模型，我们接下来可以将它们存储在支持高效相似性搜索的特殊数据结构中。

## 向量存储

LangChain [VectorStore](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) 对象包含用于将文本和 `Document` 对象添加到存储中的方法，以及使用各种相似性度量进行查询的方法。它们通常使用 [嵌入](/docs/how_to/embed_text) 模型初始化，嵌入模型决定了文本数据如何转换为数值向量。

LangChain 包括一套与不同向量存储技术的 [集成](/docs/integrations/vectorstores)。一些向量存储由提供商托管（例如，各种云提供商），需要特定凭据才能使用；一些（例如 [Postgres](/docs/integrations/vectorstores/pgvector)）运行在可以本地或通过第三方运行的单独基础设施中；其他可以在内存中运行以支持轻量级工作负载。让我们选择一个向量存储：

import VectorStoreTabs from "@theme/VectorStoreTabs";

<VectorStoreTabs/>

In [6]:
# | output: false
# | echo: false

from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

实例化向量存储后，我们现在可以索引文档。

In [7]:
ids = vector_store.add_documents(documents=all_splits)

请注意，大多数向量存储实现将允许您连接到现有的向量存储——例如，通过提供客户端、索引名称或其他信息。有关特定 [集成](/docs/integrations/vectorstores) 的更多详细信息，请参阅文档。

一旦我们实例化了包含文档的 `VectorStore`，我们就可以查询它。[VectorStore](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) 包括查询方法：
- 同步和异步；
- 按字符串查询和按向量查询；
- 是否返回相似性分数；
- 按相似性和 [最大边际相关性](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.max_marginal_relevance_search)（在查询相似性与检索结果的多样性之间取得平衡）。

这些方法通常会在其输出中包含 [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 对象的列表。

### 用法

嵌入通常将文本表示为“密集”向量，使得具有相似含义的文本在几何上接近。这使我们只需传入一个问题即可检索相关信息，而无需了解文档中使用的任何特定关键术语。

根据与字符串查询的相似性返回文档：

In [ ]:
results = vector_store.similarity_search(
    "耐克在美国有多少个配送中心？"
)

print(results[0])

page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
2023 FORM 10-K 2' metadata={'page': 4, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 3125}


异步查询：

In [ ]:
results = await vector_store.asimilarity_search("耐克是什么时候成立的？")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'page': 3, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 0}


返回分数：

In [ ]:
# 请注意，不同提供商实现了不同的分数；这里的分数
# 是一种与相似性成反比的距离度量。

results = vector_store.similarity_search_with_score("2023 年耐克的收入是多少？")
doc, score = results[0]
print(f"分数: {score}\n")
print(doc)

Score: 0.23699893057346344

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTS
The following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
•NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
•NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

根据嵌入查询的相似性返回文档：

In [ ]:
embedding = embeddings.embed_query("2023 年耐克的利润率受到了怎样的影响？")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
•Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
•Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
•Unfavorable changes in net foreign currency exchange rates, including hedges; and
•Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:' met

了解更多：

- [API 参考](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html)
- [操作指南](/docs/how_to/vectorstores)
- [特定集成文档](/docs/integrations/vectorstores)

## 检索器

LangChain `VectorStore` 对象不继承 [Runnable](https://python.langchain.com/api_reference/core/index.html#langchain-core-runnables)。LangChain [Retrievers](https://python.langchain.com/api_reference/core/index.html#langchain-core-retrievers) 是 Runnables，因此它们实现了一组标准方法（例如，同步和异步的 `invoke` 和 `batch` 操作）。虽然我们可以从向量存储构建检索器，但检索器也可以与非向量存储数据源（例如外部 API）交互。

我们可以自己创建一个简单版本，而无需继承 `Retriever`。如果我们选择希望使用的方法来检索文档，我们可以轻松创建一个 runnable。下面我们将围绕 `similarity_search` 方法构建一个：

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "耐克在美国有多少个配送中心？",
        "耐克是什么时候成立的？",
    ],
)

[[Document(metadata={'page': 4, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.\n2023 FORM 10-K 2')],
 [Document(metadata={'page': 3, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 0}, page_content='Table of Contents\nPART I\nITEM 1. BUSINESS\nGENERAL\nNIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"\n"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectivel

Vectorstores 实现了一个 `as_retriever` 方法，该方法将生成一个检索器，特别是一个 [VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html)。这些检索器包括特定的 `search_type` 和 `search_kwargs` 属性，这些属性标识调用底层向量存储的方法以及如何对其进行参数化。例如，我们可以通过以下方式复制上述内容：

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "耐克在美国有多少个配送中心？",
        "耐克是什么时候成立的？",
    ],
)

[[Document(metadata={'page': 4, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.\n2023 FORM 10-K 2')],
 [Document(metadata={'page': 3, 'source': '../example_data/nke-10k-2023.pdf', 'start_index': 0}, page_content='Table of Contents\nPART I\nITEM 1. BUSINESS\nGENERAL\nNIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"\n"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectivel

`VectorStoreRetriever` 支持 `"similarity"`（默认）、`"mmr"`（最大边际相关性，如上所述）和 `"similarity_score_threshold"` 的搜索类型。我们可以使用后者通过相似性分数对检索器输出的文档进行阈值处理。

检索器可以轻松集成到更复杂的应用程序中，例如将给定问题与检索到的上下文结合到 LLM 提示中的 [检索增强生成 (RAG)](/docs/concepts/rag) 应用程序中。要了解有关构建此类应用程序的更多信息，请查看 [RAG 教程](/docs/tutorials/rag) 教程。

### 了解更多：

检索策略可以非常丰富和复杂。例如：

- 我们可以从查询中 [推断硬规则和过滤器](/docs/how_to/self_query/)（例如，“使用 2020 年之后发布的文档”）；
- 我们可以返回与检索到的上下文以某种方式链接的 [文档](/docs/how_to/parent_document_retriever/)（例如，通过某些文档分类法）；
- 我们可以为每个上下文单元生成 [多个嵌入](/docs/how_to/multi_vector)；
- 我们可以从多个检索器 [集成结果](/docs/how_to/ensemble_retriever)；
- 我们可以为文档分配权重，例如对 [最近的文档](/docs/how_to/time_weighted_vectorstore/) 赋予更高权重。

[检索器](/docs/how_to#retrievers) 部分的操作指南涵盖了这些以及其他内置检索策略。

扩展 [BaseRetriever](https://python.langchain.com/api_reference/core/retrievers/langchain_core.retrievers.BaseRetriever.html) 类以实现自定义检索器也非常简单。请参阅我们的操作指南 [此处](/docs/how_to/custom_retriever)。


## 下一步

您现在已经了解了如何构建一个针对 PDF 文档的语义搜索引擎。

有关文档加载器的更多信息：

- [概念指南](/docs/concepts/document_loaders)
- [操作指南](/docs/how_to/#document-loaders)
- [可用集成](/docs/integrations/document_loaders/)

有关嵌入的更多信息：

- [概念指南](/docs/concepts/embedding_models/)
- [操作指南](/docs/how_to/#embedding-models)
- [可用集成](/docs/integrations/text_embedding/)

有关向量存储的更多信息：

- [概念指南](/docs/concepts/vectorstores/)
- [操作指南](/docs/how_to/#vector-stores)
- [可用集成](/docs/integrations/vectorstores/)

有关 RAG 的更多信息，请参阅：

- [构建检索增强生成 (RAG) 应用程序](/docs/tutorials/rag/)
- [相关操作指南](/docs/how_to/#qa-with-rag)